In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# set pandas to display all columns
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('new_222_data_train_norm_decorr.csv')
df_test = pd.read_csv('new_222_data_test_norm_decorr.csv')
df.sample(3)

,prev_year_access_to_exercise_opportunities,prev_year_adult_obesity,prev_year_air_pollution_particulate_matter,prev_year_alcohol_impaired_driving_deaths,prev_year_dentists,prev_year_diabetes_monitoring,prev_year_drinking_water_violations,prev_year_driving_alone_to_work,prev_year_flu_vaccinations,prev_year_injury_deaths,prev_year_long_commute_driving_alone,prev_year_mental_health_providers,prev_year_preventable_hospital_stays,prev_year_primary_care_physicians,prev_year_severe_housing_problems,prev_year_social_associations,prev_year_uninsured,prev_year_violent_crime,coastline_flag,Top_Marginal_Personal_Income_Tax_Rate,"Property_Tax_Burden_(per $1,000 of personal income)","Sales_Tax_Burden(per $1,000 of personal income)","Remaining_Tax_Burden(per $1,000 of personal income)",Estate_Inheritance_Tax_Levied_Flag,Recently_Legislated_Tax_Changes,Debt_Service_as_a_Share_of_Tax_Revenue,"Public_Employees_Per_10,000_of_Population_(full-time equivalent)","State_Liability_System_Survey_(tort litigation treatment, judicial impartiality, etc.)",State_Minimum_Wage_(federal floor is $7.25),Average Workers_Compensation_Costs_(per $100 of payroll),Tax_Expenditure_Limits,GPT_PCA_dim0,GPT_PCA_dim1,GPT_PCA_dim2,GPT_PCA_dim3,GPT_PCA_dim4,GPT_PCA_dim5,GPT_PCA_dim6,GPT_PCA_dim7,GPT_PCA_dim8,GPT_PCA_dim9,GPT_OH_Government_Support,GPT_OH_Large_Customer_Base,GPT_OH_Low_Cost_of_Living,GPT_OH_Natural_Resources,GPT_OH_Supportive_Community,GPT_OH_Limited_Financing,GPT_OH_Limited_Tech_n_Infra,GPT_OH_Low_Pop_Bad_Weather,GPT_OH_Tough_Labor_Market,GPT_OH_Unfavorable_Location,GPT_OH_Various_Downsides,pop_10yr_pct_chg,Prev_Yr_Agri_Pct_GDP,Prev_Yr_Mine_Pct_GDP,Prev_Yr_Retl_Pct_GDP,Prev_Yr_Manu_Pct_GDP,Prev_Yr_Info_Pct_GDP,Prev_Yr_FinS_Pct_GDP,Prev_Yr_EduS_Pct_GDP,Prev_Yr_EntS_Pct_GDP,Prev_Yr_Gvmt_Pct_GDP,Prev_Yr_GoTr_Pct_GDP,two_yrs_prior_pct_it_workers,health_social_pca_0,health_social_pca_1,health_social_pca_2,health_social_pca_3,health_social_pca_4,health_social_pca_5,population_2020,prev_yr_rent_50_2,gpt_ae_pca_0,gpt_ae_pca_1,gpt_ae_pca_2,microbusiness_density
2360,1.547511,-1.984667,-0.195760,-1.840561,-0.754509,0.366144,-0.400120,0.584602,1.488082,-0.299478,-0.398848,-0.696722,-0.237572,-0.096971,0.365882,2.061561,-0.262126,-0.511990,-0.291167,0.190952,0.154788,-1.282097,-0.253230,-0.590063,1.458675,-0.741886,-0.125594,0.920306,0.886771,-0.211686,-0.982727,1.383907,0.893526,-1.000320,-0.079873,-1.229370,1.972068,-1.141453,1.025512,-0.535409,-1.037130,-1.531729,-1.202197,0.219221,-0.449156,-0.585985,-1.48217,-0.889044,-0.663799,1.104564,1.55090,-0.650113,0.493633,-0.571110,-0.259022,-0.091501,-0.003782,-0.323087,0.508262,-0.183406,0.042017,0.272934,0.081107,0.894222,-1.348292,0.645307,-0.687253,-0.289679,-1.158305,0.067544,-0.237757,0.335901,-0.563614,1.080035,2.340977,5.999399
728,0.590000,-0.436031,-0.672259,-0.167603,-0.413619,1.523594,-0.816345,0.445880,-0.429633,-1.236533,-1.200022,-0.088114,-1.005131,0.698915,0.845824,-0.839301,-0.056844,-0.800083,-0.291167,-1.570762,-0.203350,1.386686,1.829377,1.694905,0.129629,0.877208,-0.326400,0.496113,2.463432,0.474472,1.138042,1.596720,1.013248,-0.174378,0.122822,-1.237793,-0.323847,-0.162517,-1.708631,0.238878,0.283065,0.652597,-1.202197,0.219221,-0.449156,-0.585985,-1.48217,-0.889044,-0.663799,1.104564,1.55090,-0.650113,1.195777,0.037485,-0.325416,0.400509,-0.752036,0.719639,-0.104502,-0.683207,0.066901,-0.025523,3.469880,-0.412783,-1.231894,-0.418612,1.469603,0.288688,1.780807,1.640825,-0.186498,0.576150,0.869664,-1.442456,0.145861,3.740335
2288,-0.106372,-0.780173,-1.446569,0.903090,-0.177618,0.696844,-0.816345,-0.663895,0.680623,0.412684,-0.078379,-0.551592,-1.517559,0.132952,-0.114061,0.396569,-0.467408,-0.213226,-0.291167,0.778190,0.371423,-0.465441,-0.645061,-0.590063,-1.099547,-1.281583,-0.788686,0.892027,-0.807349,1.050845,0.077657,-0.962402,1.417634,-1.277963,0.359156,0.527896,2.451497,-0.854128,0.401717,0.197882,-0.674299,0.652597,-1.202197,0.219221,2.225510,-0.585985,-1.48217,1.124355,1.505880,-0.904973,

In [3]:
X = df.loc[:, df.columns != 'microbusiness_density']
y = df['microbusiness_density']
y = np.log(y)

X_test = df_test.loc[:, df_test.columns != 'microbusiness_density']
y_test = df_test['microbusiness_density']
y_test = np.log(y_test)

In [4]:
# grid search for best alpha
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

lasso = Lasso()
params = {'alpha': np.logspace(-4, 1, 20)}
lasso_cv = GridSearchCV(lasso, params, cv=5)
lasso_cv.fit(X, y)
print('Best alpha:', lasso_cv.best_params_)
print('Best score:', lasso_cv.best_score_)

Best alpha: {'alpha': 0.0069519279617756054}
Best score: 0.6746772273606734


In [5]:
best_alpha = lasso_cv.best_params_['alpha']

# fit model with best alpha
lasso = Lasso(alpha=best_alpha)
lasso.fit(X, y)

from sklearn.model_selection import cross_val_score
rmse = np.sqrt(-cross_val_score(lasso, X, y, cv=5, scoring='neg_mean_squared_error'))
print('RMSE =', round(rmse.mean(),3), '±', round(rmse.std(),3))

RMSE = 0.442 ± 0.048


In [6]:
# test set
from sklearn.metrics import mean_squared_error
y_pred = lasso.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
print('RMSE test =', round(rmse_test,3))

RMSE test = 0.403


In [7]:
# print coefficients
coef = pd.DataFrame({'variable': X.columns, 'coef': lasso.coef_})
coef = coef.sort_values(by='coef', ascending=False)
coef

,variable,coef
16,prev_year_uninsured,0.088418
60,Prev_Yr_EntS_Pct_GDP,0.084689
52,pop_10yr_pct_chg,0.082150
58,Prev_Yr_FinS_Pct_GDP,0.074842
65,health_social_pca_1,0.064358
...,...,...
4,prev_year_dentists,-0.034647
61,Prev_Yr_Gvmt_Pct_GDP,-0.037717
69,health_social_pca_5,-0.058719
53,Prev_Yr_Agri_Pct_GDP,-0.068205
